In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options

In [3]:
from datetime import date
today = date.today()
print(f'Today is: {today}')

Today is: 2024-02-04


In [4]:
last_games = pd.read_excel('Models/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Models/games_odds.xlsx', 'Previous')

In [5]:
prev_games = pd.concat([prev_games, last_games])

In [6]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [7]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [8]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [9]:
table = soup.find('tbody', {'class': 'sportsbook-table__body'})
rows = table.findAll('tr')
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = american_to_decimal(int(odd))
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = american_to_decimal(int(odd))
        idx = 0
        
        games.append(game)
        game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-02-04,ORL,DET,1.387597,3.1
1,2024-02-04,PHO,WAS,1.166667,5.4


In [13]:
with pd.ExcelWriter('Models/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)